# Data Cleaning



I just added most of the imports that I used to the top of the notebooks.

In [25]:
import pandas as pd
import numpy as np
import spacy
import networkx as nx

import matplotlib.pyplot as plt
import re
import community as community_louvian
from spacy import displacy
from statistics import mean

In [ ]:
!python3 -m spacy download en_core_web_sm
!pip install nltk
!pip install textblob
!python -m textblob.download_corpora

In [17]:
nlp = spacy.load("en_core_web_sm")

#Read in data from scraper using raw github link

In [5]:
data = 'https://raw.githubusercontent.com/TheAhmir/HubermanAnalysis/main/main/scraper/data/video_metadata.csv'
data_df = pd.read_csv(data)
data_df

,video_id,video_title,video_description,video_url,video_resources,timestamps,timestamp_descriptions
0,d42b1af1-0792-4bb5-be7a-66a52060fe24,Journal Club with Dr. Peter Attia | Metformin ...,"In this journal club episode, my guest is Stan...",https://hubermanlab.com/journal-club-with-dr-p...,NaN,https://www.youtube.com/watch?v=z8c6EyMNd0A&t=...,"00:00:00 Dr. Peter Attia, Journal Club; 00:03:..."
1,0d5ce4da-c818-427d-a178-b19358276d8b,GUEST SERIES | Dr. Paul Conti: How to Understa...,This is episode 1 of a 4-part special series o...,https://hubermanlab.com/guest-series-dr-paul-c...,https://hubermanlab.com/dr-paul-conti-therapy-...,https://www.youtube.com/watch?v=tLRCS48Ens4&t=...,00:00:00 Dr. Paul Conti; 00:03:46 Sponsors: Be...
2,6da28bce-5ae5-4070-92bd-d125cdb9131c,"Marc Andreessen: How Risk Taking, Innovation &...","In this episode, my guest is Marc Andreessen, ...",https://hubermanlab.com/marc-andreessen-how-ri...,NaN,https://www.youtube.com/watch?v=yixIc1Ai6jM&t=...,00:00:00 Marc Andreessen; 00:03:02 Sponsors: L...
3,7c86e055-ff47-498d-9a9b-fcb2de1ea327,Goals Toolkit: How to Set & Achieve Your Goals,"In this episode, I describe science-based prot...",https://hubermanlab.com/goal-toolkit-how-to-se...,NaN,https://www.youtube.com/watch?v=CrtR12PBKb0&t=...,00:00:00 Goal Setting Toolkit; 00:02:09 Sponso...
4,3d7f1cea-6be4-4611-bd27-f82f2e3db411,"Dr. David Linden: Life, Death & the Neuroscien...","In this episode, my guest is Dr. David Linden,...",https://hubermanlab.com/dr-david-linden-life-d...,NaN,https://www.youtube.com/watch?v=29n0WG317tM&t=...,00:00:00 David Linden; 00:03:59 Sponsors: ROKA...
...,...,...,...,...,...,...,...
144,f85c4544-2b73-4155-8849-8916ff9625d5,"How to Defeat Jetlag, Shift Work & Sleeplessness","In this episode, I discuss a simple and reliab...",https://hubermanlab.com/find-your-temperature-...,NaN,https://www.youtube.com/watch?v=NAATB55oxeQ&t=...,00:00 Introduction; 04:15 The bedrock of sleep...
145,5c976bf9-068c-4f75-8e15-3aacf40ed71e,"Using Science to Optimize Sleep, Learning & Me...","“Office Hours” — In this episode, Dr. Huberman...",https://hubermanlab.com/using-science-to-optim...,https://academic.oup.com/biomedgerontology/art...,https://www.youtube.com/watch?v=nwSkFq4tyC0&t=...,0:00 Introduction; 5:50 Moonlight & Fire; 9:25...
146,672a51b8-0bd3-45df-872f-5db5bbade0c9,Master Your Sleep & Be More Alert When Awake,Today’s episode provides a host of information...,https://hubermanlab.com/master-your-sleep-and-...,https://www.youtube.com/watch?v=M0u9GST_j3s&t=...,https://www.youtube.com/watch?v=nm1TxQj9IsQ&t=...,0:00 Introduction; 3:30 What Is Sleep Really F...
147,076df85c-b898-47e1-85db-e5a8b0f9bc23,How Your Brain Works & Changes,Today’s episode provides an introduction to ho...,https://hubermanlab.com/how-your-nervous-syste...,NaN,https://www.youtube.com/watch?v=H-XfCl-HpRM&t=...,0:00 Introduction; 5:00 What is the Nervous Sy...


#Place YouTube IDs into Dataframe

I noticed that I wasn't able to use the video ids to retrieve the number of comments for each video using googleapiclient.discovery.

To work around this, I used selenium to do an additional scrap of each video url from my available data. I then instructed my scraper to pull, parse, and write into "urls.txt" the youtube ids from the link of each page.

In [8]:
import base64
import requests

master = "https://raw.githubusercontent.com/TheAhmir/HubermanAnalysis/main/my_dataframes/urls.txt"
req = requests.get(master)
req = req.text
id_list = req.split()

data_df['youtube_id'] = id_list

In [9]:
data_df

,video_id,video_title,video_description,video_url,video_resources,timestamps,timestamp_descriptions,youtube_id
0,d42b1af1-0792-4bb5-be7a-66a52060fe24,Journal Club with Dr. Peter Attia | Metformin ...,"In this journal club episode, my guest is Stan...",https://hubermanlab.com/journal-club-with-dr-p...,NaN,https://www.youtube.com/watch?v=z8c6EyMNd0A&t=...,"00:00:00 Dr. Peter Attia, Journal Club; 00:03:...",z8c6EyMNd0A
1,0d5ce4da-c818-427d-a178-b19358276d8b,GUEST SERIES | Dr. Paul Conti: How to Understa...,This is episode 1 of a 4-part special series o...,https://hubermanlab.com/guest-series-dr-paul-c...,https://hubermanlab.com/dr-paul-conti-therapy-...,https://www.youtube.com/watch?v=tLRCS48Ens4&t=...,00:00:00 Dr. Paul Conti; 00:03:46 Sponsors: Be...,tLRCS48Ens4
2,6da28bce-5ae5-4070-92bd-d125cdb9131c,"Marc Andreessen: How Risk Taking, Innovation &...","In this episode, my guest is Marc Andreessen, ...",https://hubermanlab.com/marc-andreessen-how-ri...,NaN,https://www.youtube.com/watch?v=yixIc1Ai6jM&t=...,00:00:00 Marc Andreessen; 00:03:02 Sponsors: L...,yixIc1Ai6jM
3,7c86e055-ff47-498d-9a9b-fcb2de1ea327,Goals Toolkit: How to Set & Achieve Your Goals,"In this episode, I describe science-based prot...",https://hubermanlab.com/goal-toolkit-how-to-se...,NaN,https://www.youtube.com/watch?v=CrtR12PBKb0&t=...,00:00:00 Goal Setting Toolkit; 00:02:09 Sponso...,CrtR12PBKb0
4,3d7f1cea-6be4-4611-bd27-f82f2e3db411,"Dr. David Linden: Life, Death & the Neuroscien...","In this episode, my guest is Dr. David Linden,...",https://hubermanlab.com/dr-david-linden-life-d...,NaN,https://www.youtube.com/watch?v=29n0WG317tM&t=...,00:00:00 David Linden; 00:03:59 Sponsors: ROKA...,29n0WG317tM
...,...,...,...,...,...,...,...,...
144,f85c4544-2b73-4155-8849-8916ff9625d5,"How to Defeat Jetlag, Shift Work & Sleeplessness","In this episode, I discuss a simple and reliab...",https://hubermanlab.com/find-your-temperature-...,NaN,https://www.youtube.com/watch?v=NAATB55oxeQ&t=...,00:00 Introduction; 04:15 The bedrock of sleep...,NAATB55oxeQ
145,5c976bf9-068c-4f75-8e15-3aacf40ed71e,"Using Science to Optimize Sleep, Learning & Me...","“Office Hours” — In this episode, Dr. Huberman...",https://hubermanlab.com/using-science-to-optim...,https://academic.oup.com/biomedgerontology/art...,https://www.youtube.com/watch?v=nwSkFq4tyC0&t=...,0:00 Introduction; 5:50 Moonlight & Fire; 9:25...,nwSkFq4tyC0
146,672a51b8-0bd3-45df-872f-5db5bbade0c9,Master Your Sleep & Be More Alert When Awake,Today’s episode provides a host of information...,https://hubermanlab.com/master-your-sleep-and-...,https://www.youtube.com/watch?v=M0u9GST_j3s&t=...,https://www.youtube.com/watch?v=nm1TxQj9IsQ&t=...,0:00 Introduction; 3:30 What Is Sleep Really F...,nm1TxQj9IsQ
147,076df85c-b898-47e1-85db-e5a8b0f9bc23,How Your Brain Works & Changes,Today’s episode provides an introduction to ho...,https://hubermanlab.com/how-your-nervous-syste...,NaN,https://www.youtube.com/watch?v=H-XfCl-HpRM&t=...,0:00 Introduction; 5:00 What is the Nervous Sy...,H-XfCl-HpRM


In [10]:
data_df.drop(data_df.tail(1).index,inplace=True)
data_df

,video_id,video_title,video_description,video_url,video_resources,timestamps,timestamp_descriptions,youtube_id
0,d42b1af1-0792-4bb5-be7a-66a52060fe24,Journal Club with Dr. Peter Attia | Metformin ...,"In this journal club episode, my guest is Stan...",https://hubermanlab.com/journal-club-with-dr-p...,NaN,https://www.youtube.com/watch?v=z8c6EyMNd0A&t=...,"00:00:00 Dr. Peter Attia, Journal Club; 00:03:...",z8c6EyMNd0A
1,0d5ce4da-c818-427d-a178-b19358276d8b,GUEST SERIES | Dr. Paul Conti: How to Understa...,This is episode 1 of a 4-part special series o...,https://hubermanlab.com/guest-series-dr-paul-c...,https://hubermanlab.com/dr-paul-conti-therapy-...,https://www.youtube.com/watch?v=tLRCS48Ens4&t=...,00:00:00 Dr. Paul Conti; 00:03:46 Sponsors: Be...,tLRCS48Ens4
2,6da28bce-5ae5-4070-92bd-d125cdb9131c,"Marc Andreessen: How Risk Taking, Innovation &...","In this episode, my guest is Marc Andreessen, ...",https://hubermanlab.com/marc-andreessen-how-ri...,NaN,https://www.youtube.com/watch?v=yixIc1Ai6jM&t=...,00:00:00 Marc Andreessen; 00:03:02 Sponsors: L...,yixIc1Ai6jM
3,7c86e055-ff47-498d-9a9b-fcb2de1ea327,Goals Toolkit: How to Set & Achieve Your Goals,"In this episode, I describe science-based prot...",https://hubermanlab.com/goal-toolkit-how-to-se...,NaN,https://www.youtube.com/watch?v=CrtR12PBKb0&t=...,00:00:00 Goal Setting Toolkit; 00:02:09 Sponso...,CrtR12PBKb0
4,3d7f1cea-6be4-4611-bd27-f82f2e3db411,"Dr. David Linden: Life, Death & the Neuroscien...","In this episode, my guest is Dr. David Linden,...",https://hubermanlab.com/dr-david-linden-life-d...,NaN,https://www.youtube.com/watch?v=29n0WG317tM&t=...,00:00:00 David Linden; 00:03:59 Sponsors: ROKA...,29n0WG317tM
...,...,...,...,...,...,...,...,...
143,1f8e45d8-6c6f-4903-9df5-23896c087682,Understanding and Using Dreams to Learn and to...,This episode is all about the two major kinds ...,https://hubermanlab.com/understanding-and-usin...,NaN,https://www.youtube.com/watch?v=FFwA0QFmpQ4&t=...,00:00 Introduction; 03:00 The Dream Mask; 06:0...,FFwA0QFmpQ4
144,f85c4544-2b73-4155-8849-8916ff9625d5,"How to Defeat Jetlag, Shift Work & Sleeplessness","In this episode, I discuss a simple and reliab...",https://hubermanlab.com/find-your-temperature-...,NaN,https://www.youtube.com/watch?v=NAATB55oxeQ&t=...,00:00 Introduction; 04:15 The bedrock of sleep...,NAATB55oxeQ
145,5c976bf9-068c-4f75-8e15-3aacf40ed71e,"Using Science to Optimize Sleep, Learning & Me...","“Office Hours” — In this episode, Dr. Huberman...",https://hubermanlab.com/using-science-to-optim...,https://academic.oup.com/biomedgerontology/art...,https://www.youtube.com/watch?v=nwSkFq4tyC0&t=...,0:00 Introduction; 5:50 Moonlight & Fire; 9:25...,nwSkFq4tyC0
146,672a51b8-0bd3-45df-872f-5db5bbade0c9,Master Your Sleep & Be More Alert When Awake,Today’s episode provides a host of information...,https://hubermanlab.com/master-your-sleep-and-...,https://www.youtube.com/watch?v=M0u9GST_j3s&t=...,https://www.youtube.com/watch?v=nm1TxQj9IsQ&t=...,0:00 Introduction; 3:30 What Is Sleep Really F...,nm1TxQj9IsQ


#Install Google Api Client and Write Functions to Pull Comments from Youtube IDs

I used StackOverflow to find a function to extract the number of comments for each video.

This function reviewed each individual comment, so it ended up hitting the daily limit of free api use with my api key. However, I found out that I could just make a second api key to extract all this data in one day.

In [ ]:
!pip install --upgrade google-api-python-client
!pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [ ]:
from googleapiclient.discovery import build

# recursive function to get all replies in a comment thread
def get_replies(comment_id, token):
    replies_response = yt_object.comments().list(part = 'snippet', maxResults = 100, parentId = comment_id, pageToken = token).execute()

    for reply in replies_response['items']:
        all_comments.append(reply['snippet']['textDisplay'])

    if replies_response.get("nextPageToken"):
        return get_replies(comment_id, replies_response['nextPageToken'])
    else:
        return []


# recursive function to get all comments
def get_comments(youtube, video_id, next_view_token):
    global all_comments

    # check for token
    if len(next_view_token.strip()) == 0:
        all_comments = []

    if next_view_token == '':
        # get the initial response
        comment_list = youtube.commentThreads().list(part = 'snippet', maxResults = 100, videoId = video_id, order = 'relevance').execute()
    else:
        # get the next page response
        comment_list = youtube.commentThreads().list(part = 'snippet', maxResults = 100, videoId = video_id, order='relevance', pageToken=next_view_token).execute()
    # loop through all top level comments
    for comment in comment_list['items']:
        # add comment to list
        all_comments.append([comment['snippet']['topLevelComment']['snippet']['textDisplay']])
        # get number of replies
        reply_count = comment['snippet']['totalReplyCount']
        all_replies = []
        # if replies greater than 0
        if reply_count > 0:
            # get first 100 replies
            replies_list = youtube.comments().list(part='snippet', maxResults=100, parentId=comment['id']).execute()
            for reply in replies_list['items']:
                # add reply to list
                all_replies.append(reply['snippet']['textDisplay'])

            # check for more replies
            while "nextPageToken" in replies_list:
                token_reply = replies_list['nextPageToken']
                # get next set of 100 replies
                replies_list = youtube.comments().list(part = 'snippet', maxResults = 100, parentId = comment['id'], pageToken = token_reply).execute()
                for reply in replies_list['items']:
                    # add reply to list
                    all_replies.append(reply['snippet']['textDisplay'])

        # add all replies to the comment
        all_comments[-1].append(all_replies)

    if "nextPageToken" in comment_list:
        return get_comments(youtube, video_id, comment_list['nextPageToken'])
    else:
        return []



### Next, I added the full list of number of comments into the original dataset.

In [ ]:
num_comments = []

In [ ]:
for i in data_df.loc[len(num_comments):,"youtube_id"]:
  video_id = i
  api_key = 'XXXXXX'#api key redacted
  all_comments = []

  # build a youtube object using our api key
  yt_object = build('youtube', 'v3', developerKey=api_key)

  # get all comments and replies
  comments = get_comments(yt_object, video_id, '')

  num_comments.append(len(all_comments))

In [ ]:
data_df["num_comments"] = num_comments

In [ ]:
data_df.to_csv('final_data.csv', index=False)

In [11]:
data = 'https://raw.githubusercontent.com/TheAhmir/HubermanAnalysis/main/my_dataframes/final_data.csv'
data_df = pd.read_csv(data)
data_df

,video_id,video_title,video_description,video_url,video_resources,timestamps,timestamp_descriptions,youtube_id,num_comments
0,d42b1af1-0792-4bb5-be7a-66a52060fe24,Journal Club with Dr. Peter Attia | Metformin ...,"In this journal club episode, my guest is Stan...",https://hubermanlab.com/journal-club-with-dr-p...,NaN,https://www.youtube.com/watch?v=z8c6EyMNd0A&t=...,"00:00:00 Dr. Peter Attia, Journal Club; 00:03:...",z8c6EyMNd0A,318
1,0d5ce4da-c818-427d-a178-b19358276d8b,GUEST SERIES | Dr. Paul Conti: How to Understa...,This is episode 1 of a 4-part special series o...,https://hubermanlab.com/guest-series-dr-paul-c...,https://hubermanlab.com/dr-paul-conti-therapy-...,https://www.youtube.com/watch?v=tLRCS48Ens4&t=...,00:00:00 Dr. Paul Conti; 00:03:46 Sponsors: Be...,tLRCS48Ens4,946
2,6da28bce-5ae5-4070-92bd-d125cdb9131c,"Marc Andreessen: How Risk Taking, Innovation &...","In this episode, my guest is Marc Andreessen, ...",https://hubermanlab.com/marc-andreessen-how-ri...,NaN,https://www.youtube.com/watch?v=yixIc1Ai6jM&t=...,00:00:00 Marc Andreessen; 00:03:02 Sponsors: L...,yixIc1Ai6jM,262
3,7c86e055-ff47-498d-9a9b-fcb2de1ea327,Goals Toolkit: How to Set & Achieve Your Goals,"In this episode, I describe science-based prot...",https://hubermanlab.com/goal-toolkit-how-to-se...,NaN,https://www.youtube.com/watch?v=CrtR12PBKb0&t=...,00:00:00 Goal Setting Toolkit; 00:02:09 Sponso...,CrtR12PBKb0,539
4,3d7f1cea-6be4-4611-bd27-f82f2e3db411,"Dr. David Linden: Life, Death & the Neuroscien...","In this episode, my guest is Dr. David Linden,...",https://hubermanlab.com/dr-david-linden-life-d...,NaN,https://www.youtube.com/watch?v=29n0WG317tM&t=...,00:00:00 David Linden; 00:03:59 Sponsors: ROKA...,29n0WG317tM,497
...,...,...,...,...,...,...,...,...,...
143,1f8e45d8-6c6f-4903-9df5-23896c087682,Understanding and Using Dreams to Learn and to...,This episode is all about the two major kinds ...,https://hubermanlab.com/understanding-and-usin...,NaN,https://www.youtube.com/watch?v=FFwA0QFmpQ4&t=...,00:00 Introduction; 03:00 The Dream Mask; 06:0...,FFwA0QFmpQ4,1070
144,f85c4544-2b73-4155-8849-8916ff9625d5,"How to Defeat Jetlag, Shift Work & Sleeplessness","In this episode, I discuss a simple and reliab...",https://hubermanlab.com/find-your-temperature-...,NaN,https://www.youtube.com/watch?v=NAATB55oxeQ&t=...,00:00 Introduction; 04:15 The bedrock of sleep...,NAATB55oxeQ,990
145,5c976bf9-068c-4f75-8e15-3aacf40ed71e,"Using Science to Optimize Sleep, Learning & Me...","“Office Hours” — In this episode, Dr. Huberman...",https://hubermanlab.com/using-science-to-optim...,https://academic.oup.com/biomedgerontology/art...,https://www.youtube.com/watch?v=nwSkFq4tyC0&t=...,0:00 Introduction; 5:50 Moonlight & Fire; 9:25...,nwSkFq4tyC0,1085
146,672a51b8-0bd3-45df-872f-5db5bbade0c9,Master Your Sleep & Be More Alert When Awake,Today’s episode provides a host of information...,https://hubermanlab.com/master-your-sleep-and-...,https://www.youtube.com/watch?v=M0u9GST_j3s&t=...,https://www.youtube.com/watch?v=nm1TxQj9IsQ&t=...,0:00 Introduction; 3:30 What Is Sleep Really F...,nm1TxQj9IsQ,1196


### As the code to retrieve the comments was running, I thought it'd be interesting to try and see the relationship of different topics used by Huberman.

To do this, I first had to figure out the topic of each video from the description and title. The method I chose to use for figuring out the topic of the video was looking at any noun phrases throughout the title and description.

Using the NLTK library was the first method I tried in order to locate noun phrases. Once again I went to StackOverflow to find a suitable function. I don't have much experience using some libraries so StackOverflow was a life saver a few times throughout this project.

I found this option gave pretty solid result as far as potential topics, however, it seemed to be unable to determine any topic for some of the data.

In [23]:
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
import pandas as pd

# Defining a grammar & Parser
NP = "NP: {(<V\w+>|<NN\w?>)+.*<NN\w?>}"
chunker = RegexpParser(NP)

def get_continuous_chunks(text, chunk_func=ne_chunk):
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    return continuous_chunk


topics = data_df['video_title'].apply(lambda sent: get_continuous_chunks(sent, chunker.parse))
print(data_df.loc[3, 'video_title'])
print(topics[3])
print()
for i in topics[3]:
  doc = nlp(i)
  etypes = [tok.ent_type_ for tok in doc]
  if 'PERSON' in etypes:
    print(doc)

Goals Toolkit: How to Set & Achieve Your Goals
['Goals Toolkit']



I then learned that Spacy has a function that determines noun phrases called "noun_chunks". This function yielded less ideal results than NLTK, but I found that it could come up with results for most if not all of the data I had.

In [24]:
txt = data_df.loc[3, 'video_title']
blob = nlp(txt)
print(txt)
list(blob.noun_chunks)

Goals Toolkit: How to Set & Achieve Your Goals


[Goals Toolkit, Set, Your Goals]

### My next idea was to use the noun phrases found by both of the previously mentioned methods instead of picking one.

To do this, I combined the results for each of the methods for each video. I then took out each phrase that Spacy concluded contains a person's name.

I did this because I believe any human not previously exposed to the names of the guest stars would not be able to asertain the topic of the video from them.

In [28]:
noun_phrases = []
topics = data_df['video_title'].apply(lambda sent: get_continuous_chunks(sent, chunker.parse))
for i in range(len(topics)):
  txt = data_df.loc[i, 'video_title']
  blob = nlp(txt)
  spacy_phrases = [str(x) for x in blob.noun_chunks]
  cur_phrases = spacy_phrases + topics[i]
  removal = []
  for x in cur_phrases:
    doc = nlp(str(x))
    etypes = [tok.ent_type_ for tok in doc]
    if 'PERSON' in etypes or "GUEST SERIES" in str(x):
      removal.append(x)
  final_phrases = [str(x) for x in cur_phrases if x not in removal]
  noun_phrases.append(final_phrases)

In [29]:
len(noun_phrases)

148

In [30]:
data_df['noun_phrases'] = noun_phrases

I then decided that it wouldn't hurt to pull the noun phrases from each of the descriptions as well.

In [32]:
noun_phrases = []
topics = data_df['video_description'].apply(lambda sent: get_continuous_chunks(sent, chunker.parse))
for i in range(len(topics)):
  txt = data_df.loc[i, 'video_description']
  blob = nlp(txt)
  spacy_phrases = [str(x) for x in blob.noun_chunks]
  cur_phrases = spacy_phrases + topics[i]
  removal = []
  for x in cur_phrases:
    doc = nlp(str(x))
    etypes = [tok.ent_type_ for tok in doc]
    if 'PERSON' in etypes or "GUEST SERIES" in str(x):
      removal.append(x)
  final_phrases = [str(x) for x in cur_phrases if x not in removal]
  noun_phrases.append(final_phrases)

In [33]:
data_df['noun_phrases(desc)'] = noun_phrases

In [ ]:
data_df.to_csv('final2_data.csv', index=False)

In [43]:
data = 'https://raw.githubusercontent.com/TheAhmir/HubermanAnalysis/main/my_dataframes/final2_data.csv'
data_df = pd.read_csv(data)
data_df

,video_id,video_title,video_description,video_url,video_resources,timestamps,timestamp_descriptions,youtube_id,num_comments,noun_phrases,noun_phrases(desc)
0,d42b1af1-0792-4bb5-be7a-66a52060fe24,Journal Club with Dr. Peter Attia | Metformin ...,"In this journal club episode, my guest is Stan...",https://hubermanlab.com/journal-club-with-dr-p...,NaN,https://www.youtube.com/watch?v=z8c6EyMNd0A&t=...,"00:00:00 Dr. Peter Attia, Journal Club; 00:03:...",z8c6EyMNd0A,318,"['Journal Club', 'Longevity', 'The Power', 'Be...","['this journal club episode', 'my guest', 'Sta..."
1,0d5ce4da-c818-427d-a178-b19358276d8b,GUEST SERIES | Dr. Paul Conti: How to Understa...,This is episode 1 of a 4-part special series o...,https://hubermanlab.com/guest-series-dr-paul-c...,https://hubermanlab.com/dr-paul-conti-therapy-...,https://www.youtube.com/watch?v=tLRCS48Ens4&t=...,00:00:00 Dr. Paul Conti; 00:03:46 Sponsors: Be...,tLRCS48Ens4,946,"['Understand', 'Assess', 'Your Mental Health']","['This', 'episode', 'a 4-part special series',..."
2,6da28bce-5ae5-4070-92bd-d125cdb9131c,"Marc Andreessen: How Risk Taking, Innovation &...","In this episode, my guest is Marc Andreessen, ...",https://hubermanlab.com/marc-andreessen-how-ri...,NaN,https://www.youtube.com/watch?v=yixIc1Ai6jM&t=...,00:00:00 Marc Andreessen; 00:03:02 Sponsors: L...,yixIc1Ai6jM,262,"['How Risk Taking', 'Innovation', 'Artificial ...","['this episode', 'my guest', 'the legendary so..."
3,7c86e055-ff47-498d-9a9b-fcb2de1ea327,Goals Toolkit: How to Set & Achieve Your Goals,"In this episode, I describe science-based prot...",https://hubermanlab.com/goal-toolkit-how-to-se...,NaN,https://www.youtube.com/watch?v=CrtR12PBKb0&t=...,00:00:00 Goal Setting Toolkit; 00:02:09 Sponso...,CrtR12PBKb0,539,"['Goals Toolkit', 'Set', 'Your Goals', 'Goals ...","['this episode', 'I', 'science-based protocols..."
4,3d7f1cea-6be4-4611-bd27-f82f2e3db411,"Dr. David Linden: Life, Death & the Neuroscien...","In this episode, my guest is Dr. David Linden,...",https://hubermanlab.com/dr-david-linden-life-d...,NaN,https://www.youtube.com/watch?v=29n0WG317tM&t=...,00:00:00 David Linden; 00:03:59 Sponsors: ROKA...,29n0WG317tM,497,"['Life', 'Death', 'the Neuroscience', 'Your Un...","['this episode', 'my guest', 'Ph.D.', 'profess..."
...,...,...,...,...,...,...,...,...,...,...,...
143,1f8e45d8-6c6f-4903-9df5-23896c087682,Understanding and Using Dreams to Learn and to...,This episode is all about the two major kinds ...,https://hubermanlab.com/understanding-and-usin...,NaN,https://www.youtube.com/watch?v=FFwA0QFmpQ4&t=...,00:00 Introduction; 03:00 The Dream Mask; 06:0...,FFwA0QFmpQ4,1070,"['Dreams', 'Using Dreams']","['This episode', 'the two major kinds', 'dream..."
144,f85c4544-2b73-4155-8849-8916ff9625d5,"How to Defeat Jetlag, Shift Work & Sleeplessness","In this episode, I discuss a simple and reliab...",https://hubermanlab.com/find-your-temperature-...,NaN,https://www.youtube.com/watch?v=NAATB55oxeQ&t=...,00:00 Introduction; 04:15 The bedrock of sleep...,NAATB55oxeQ,990,"['How to Defeat', 'Jetlag', 'Shift Work', 'Sle...","['this episode', 'I', 'a simple and reliable m..."
145,5c976bf9-068c-4f75-8e15-3aacf40ed71e,"Using Science to Optimize Sleep, Learning & Me...","“Office Hours” — In this episode, Dr. Huberman...",https://hubermanlab.com/using-science-to-optim...,https://academic.oup.com/biomedgerontology/art...,https://www.youtube.com/watch?v=nwSkFq4tyC0&t=...,0:00 Introduction; 5:50 Moonlight & Fire; 9:25...,nwSkFq4tyC0,1085,"['Science', 'Optimize Sleep', 'Learning', 'Met...","['this episode', 'your most commonly asked que..."
146,672a51b8-0bd3-45df-872f-5db5bbade0c9,Master Your Sleep & Be More Alert When Awake,Today’s episode provides a host of information...,https://hubermanlab.com/master-your-sleep-and-...,https://www.youtube.com/watch?v=M0u9GST_j3s&t=...,https://www.youtube.com/watch?v=nm1TxQj9IsQ&t=...,0:00 Introduction; 3:30 What Is Sleep Really F...,nm1TxQj9IsQ,1196,"['Master Your Sleep', 'Master Your Sleep']","[

In [36]:
print(data_df.loc[1, 'noun_phrases'])
print(data_df.loc[140, 'noun_phrases'])

['Understand', 'Assess', 'Your Mental Health']
['Your Learning', 'Creativity', 'Science-based Tools', 'Optimize Your Learning']


###Text Similarities

Next, I wanted to calculate the text similarities between the videos. My approach was that higher similarity scores of two sets of noun phrases would mean that two videos should be close in topic.

To calculate text similarity I used TFID and cosine similarity of each videos title and description concatenated together

I again had to visit StackOverflow for this function and made adjustments to work with my data the way I wanted it to. It turns out we worked with TFID in class just after I finished this part, so it was interesting having more explanation of how it could be used.

In [38]:
# noun phrase similarity function

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def similarity(index1, index2):
  # Convert the texts into TF-IDF vectors
  vectorizer = TfidfVectorizer()
  vectors1 = vectorizer.fit_transform([' '.join(data_df.loc[index1, 'noun_phrases']), ' '.join(data_df.loc[index2, 'noun_phrases'])])
  vectors2 = vectorizer.fit_transform([' '.join(data_df.loc[index1, 'noun_phrases(desc)']), ' '.join(data_df.loc[index2, 'noun_phrases(desc)'])])

  # Calculate the cosine similarity between the vectors
  similarity1 = cosine_similarity(vectors1)
  similarity2 = cosine_similarity(vectors2)
  return similarity1[0][1] + similarity2[0][1]

In [40]:
relationships = []
rows = list(range(len(data_df.index)))
for i in range(len(data_df.index)):
    for x in range(len(data_df.index)):
      if i != x:
        try:
          value = similarity(i,x)
        except:
          value = 0
        relationships.append({'source': data_df.loc[i,'video_title'], 'target':data_df.loc[x,'video_title'], 'similarity_score': value})

To keep track of any data I obtained, I saved the relationships into dataframes to commit to my github repository.

In [ ]:
relationship_df = pd.DataFrame(relationships)
relationship_df.to_csv('relationships.csv', index=False)

In [41]:
relationships = 'https://raw.githubusercontent.com/TheAhmir/HubermanAnalysis/main/my_dataframes/relationships.csv'
relationship_df = pd.read_csv(relationships)
relationship_df

,source,target,similarity_score
0,Journal Club with Dr. Peter Attia | Metformin ...,GUEST SERIES | Dr. Paul Conti: How to Understa...,0.227811
1,Journal Club with Dr. Peter Attia | Metformin ...,"Marc Andreessen: How Risk Taking, Innovation &...",0.223216
2,Journal Club with Dr. Peter Attia | Metformin ...,Goals Toolkit: How to Set & Achieve Your Goals,0.086179
3,Journal Club with Dr. Peter Attia | Metformin ...,"Dr. David Linden: Life, Death & the Neuroscien...",0.410669
4,Journal Club with Dr. Peter Attia | Metformin ...,Dr. Rena Malik: Improving Sexual & Urological ...,0.144752
...,...,...,...
21751,How Your Brain Works & Changes,How to Focus to Change Your Brain,0.830417
21752,How Your Brain Works & Changes,Understanding and Using Dreams to Learn and to...,0.101601
21753,How Your Brain Works & Changes,"How to Defeat Jetlag, Shift Work & Sleeplessness",0.172682
21754,How Your Brain Works & Changes,"Using Science to Optimize Sleep, Learning & Me...",0.124961


### Finally I wanted to pull the number of views for each video

It should be noted that while doing this, at some point, I noticed that the last row in my data was a podcast rather than a YouTube video. I decided to remove that row and continue my analysis on just the YouTube videos going forward.

I again had to resort to StackOverflow for this function...

In [46]:
import json
def youtube_view_count_shortcode(params):
    video_id = params['id']  # view id here
    api_key = 'XXXXXX'  # Replace with your API key
    url = f"https://www.googleapis.com/youtube/v3/videos?part=statistics&id={video_id}&key={api_key}"

    response = requests.get(url)
    if response.status_code == 200:
        data = json.loads(response.text)
        views = data['items'][0]['statistics']['viewCount']
        return '{:,}'.format(int(views))
    else:
        return "Error fetching data"


In [47]:
params = {'id': 'z8c6EyMNd0A'}
result = youtube_view_count_shortcode(params)
print(result)

190,672


It seemed that the view counts found using my api key were printing out as strings with commas in them, so i converted each of them into floats for easier use when visualizing the data.

In [ ]:
view_count = []
for i in data_df.loc[:,'youtube_id']:
  params = {'id': i}
  result = youtube_view_count_shortcode(params)
  value_as_float = float(result.replace(",", ""))
  view_count.append(value_as_float)

In [ ]:
data_df['view_count'] = view_count

In [ ]:
data_df.to_csv('final3_data.csv', index=False)

In [48]:
data = 'https://raw.githubusercontent.com/TheAhmir/HubermanAnalysis/main/my_dataframes/final3_data.csv'
data_df = pd.read_csv(data)
data_df

,video_id,video_title,video_description,video_url,video_resources,timestamps,timestamp_descriptions,youtube_id,num_comments,noun_phrases,noun_phrases(desc),view_count
0,d42b1af1-0792-4bb5-be7a-66a52060fe24,Journal Club with Dr. Peter Attia | Metformin ...,"In this journal club episode, my guest is Stan...",https://hubermanlab.com/journal-club-with-dr-p...,NaN,https://www.youtube.com/watch?v=z8c6EyMNd0A&t=...,"00:00:00 Dr. Peter Attia, Journal Club; 00:03:...",z8c6EyMNd0A,318,"['Journal Club', 'Longevity', 'The Power', 'Be...","['this journal club episode', 'my guest', 'Sta...","190,546"
1,0d5ce4da-c818-427d-a178-b19358276d8b,GUEST SERIES | Dr. Paul Conti: How to Understa...,This is episode 1 of a 4-part special series o...,https://hubermanlab.com/guest-series-dr-paul-c...,https://hubermanlab.com/dr-paul-conti-therapy-...,https://www.youtube.com/watch?v=tLRCS48Ens4&t=...,00:00:00 Dr. Paul Conti; 00:03:46 Sponsors: Be...,tLRCS48Ens4,946,"['Understand', 'Assess', 'Your Mental Health']","['This', 'episode', 'a 4-part special series',...","1,623,691"
2,6da28bce-5ae5-4070-92bd-d125cdb9131c,"Marc Andreessen: How Risk Taking, Innovation &...","In this episode, my guest is Marc Andreessen, ...",https://hubermanlab.com/marc-andreessen-how-ri...,NaN,https://www.youtube.com/watch?v=yixIc1Ai6jM&t=...,00:00:00 Marc Andreessen; 00:03:02 Sponsors: L...,yixIc1Ai6jM,262,"['How Risk Taking', 'Innovation', 'Artificial ...","['this episode', 'my guest', 'the legendary so...","125,192"
3,7c86e055-ff47-498d-9a9b-fcb2de1ea327,Goals Toolkit: How to Set & Achieve Your Goals,"In this episode, I describe science-based prot...",https://hubermanlab.com/goal-toolkit-how-to-se...,NaN,https://www.youtube.com/watch?v=CrtR12PBKb0&t=...,00:00:00 Goal Setting Toolkit; 00:02:09 Sponso...,CrtR12PBKb0,539,"['Goals Toolkit', 'Set', 'Your Goals', 'Goals ...","['this episode', 'I', 'science-based protocols...","464,095"
4,3d7f1cea-6be4-4611-bd27-f82f2e3db411,"Dr. David Linden: Life, Death & the Neuroscien...","In this episode, my guest is Dr. David Linden,...",https://hubermanlab.com/dr-david-linden-life-d...,NaN,https://www.youtube.com/watch?v=29n0WG317tM&t=...,00:00:00 David Linden; 00:03:59 Sponsors: ROKA...,29n0WG317tM,497,"['Life', 'Death', 'the Neuroscience', 'Your Un...","['this episode', 'my guest', 'Ph.D.', 'profess...","259,554"
...,...,...,...,...,...,...,...,...,...,...,...,...
143,1f8e45d8-6c6f-4903-9df5-23896c087682,Understanding and Using Dreams to Learn and to...,This episode is all about the two major kinds ...,https://hubermanlab.com/understanding-and-usin...,NaN,https://www.youtube.com/watch?v=FFwA0QFmpQ4&t=...,00:00 Introduction; 03:00 The Dream Mask; 06:0...,FFwA0QFmpQ4,1070,"['Dreams', 'Using Dreams']","['This episode', 'the two major kinds', 'dream...","786,733"
144,f85c4544-2b73-4155-8849-8916ff9625d5,"How to Defeat Jetlag, Shift Work & Sleeplessness","In this episode, I discuss a simple and reliab...",https://hubermanlab.com/find-your-temperature-...,NaN,https://www.youtube.com/watch?v=NAATB55oxeQ&t=...,00:00 Introduction; 04:15 The bedrock of sleep...,NAATB55oxeQ,990,"['How to Defeat', 'Jetlag', 'Shift Work', 'Sle...","['this episode', 'I', 'a simple and reliable m...","420,764"
145,5c976bf9-068c-4f75-8e15-3aacf40ed71e,"Using Science to Optimize Sleep, Learning & Me...","“Office Hours” — In this episode, Dr. Huberman...",https://hubermanlab.com/using-science-to-optim...,https://academic.oup.com/biomedgerontology/art...,https://www.youtube.com/watch?v=nwSkFq4tyC0&t=...,0:00 Introduction; 5:50 Moonlight & Fire; 9:25...,nwSkFq4tyC0,1085,"['Science', 'Optimize Sleep', 'Learning', 'Met...","['this episode', 'your most commonly asked que...","1,203,178"
146,672a51b8-0bd3-45df-872f-5db5bbade0c9,Master Your Sleep & Be More Alert When Awake,Today’s episode provides a host of information...,https://hubermanlab.com/master-your-sleep-and-...,https://www.youtube.com/watch?v=M0u9GST_j3s&t=...,https://www.youtube.com/watch?v=nm1TxQj9IsQ&t=...,0:00 Introducti

###Removing Unnecessary Data

At this point, I determined that the some of the originally provided columns were not going to be used for my visualization and further cleaned the data by removing them.

In [49]:
cleaned_data = data_df
cleaned_data = cleaned_data.drop(['video_id', 'video_resources', 'timestamps', 'timestamp_descriptions'], axis=1)

### My data was finally ready to be used in visualizations!

In [50]:
cleaned_data

,video_title,video_description,video_url,youtube_id,num_comments,noun_phrases,noun_phrases(desc),view_count
0,Journal Club with Dr. Peter Attia | Metformin ...,"In this journal club episode, my guest is Stan...",https://hubermanlab.com/journal-club-with-dr-p...,z8c6EyMNd0A,318,"['Journal Club', 'Longevity', 'The Power', 'Be...","['this journal club episode', 'my guest', 'Sta...","190,546"
1,GUEST SERIES | Dr. Paul Conti: How to Understa...,This is episode 1 of a 4-part special series o...,https://hubermanlab.com/guest-series-dr-paul-c...,tLRCS48Ens4,946,"['Understand', 'Assess', 'Your Mental Health']","['This', 'episode', 'a 4-part special series',...","1,623,691"
2,"Marc Andreessen: How Risk Taking, Innovation &...","In this episode, my guest is Marc Andreessen, ...",https://hubermanlab.com/marc-andreessen-how-ri...,yixIc1Ai6jM,262,"['How Risk Taking', 'Innovation', 'Artificial ...","['this episode', 'my guest', 'the legendary so...","125,192"
3,Goals Toolkit: How to Set & Achieve Your Goals,"In this episode, I describe science-based prot...",https://hubermanlab.com/goal-toolkit-how-to-se...,CrtR12PBKb0,539,"['Goals Toolkit', 'Set', 'Your Goals', 'Goals ...","['this episode', 'I', 'science-based protocols...","464,095"
4,"Dr. David Linden: Life, Death & the Neuroscien...","In this episode, my guest is Dr. David Linden,...",https://hubermanlab.com/dr-david-linden-life-d...,29n0WG317tM,497,"['Life', 'Death', 'the Neuroscience', 'Your Un...","['this episode', 'my guest', 'Ph.D.', 'profess...","259,554"
...,...,...,...,...,...,...,...,...
143,Understanding and Using Dreams to Learn and to...,This episode is all about the two major kinds ...,https://hubermanlab.com/understanding-and-usin...,FFwA0QFmpQ4,1070,"['Dreams', 'Using Dreams']","['This episode', 'the two major kinds', 'dream...","786,733"
144,"How to Defeat Jetlag, Shift Work & Sleeplessness","In this episode, I discuss a simple and reliab...",https://hubermanlab.com/find-your-temperature-...,NAATB55oxeQ,990,"['How to Defeat', 'Jetlag', 'Shift Work', 'Sle...","['this episode', 'I', 'a simple and reliable m...","420,764"
145,"Using Science to Optimize Sleep, Learning & Me...","“Office Hours” — In this episode, Dr. Huberman...",https://hubermanlab.com/using-science-to-optim...,nwSkFq4tyC0,1085,"['Science', 'Optimize Sleep', 'Learning', 'Met...","['this episode', 'your most commonly asked que...","1,203,178"
146,Master Your Sleep & Be More Alert When Awake,Today’s episode provides a host of information...,https://hubermanlab.com/master-your-sleep-and-...,nm1TxQj9IsQ,1196,"['Master Your Sleep', 'Master Your Sleep']","['Today’s episode', 'a host', 'information', '...","3,023,121"


In [ ]:
cleaned_data.to_csv('cleaned_data.csv', index=False)